# Homework 1: CS-GY 9223
## Exploring 20 NewsGroups

In this homework, you will write a D3 Visualization in Javascript and integrate it in Jupyter Notebook.

The goal of this exercise is to explore the **20 News Groups dataset**, a popular machine learning dataset that contains news articles grouped in 20 topics. Your visualization should receive the dataset and display a bar chart with the top most frequent words in the dataset. The user should be able to filter the data based on topic (for example, by clicking in checkboxes, selecting from a drop down menu, etc.). The user should also be able to export the selected documents from the selected topic back to Python (using a button).

In summary, your visualization should have the following capabilities:
- Display a bar chart with the top K words in the document collection
- Enable the user to filter the documents based on topic, and display a bar chart with the frequency of the top K words from that topic.
- Export the documents from the selected news topic back to python (as a list of strings).
- The visualization has to be integrated with python. The API should have two functions:
  - `plot_top_words(documents, K) # plot top K words using D3 and Javascript`
  - `get_exported_documents() # get the exported documents back to python`
  
Example of the resulting visualization:
<img src="https://github.com/yeb2Binfang/CS_9223_Visualization_for_ML/blob/main/HW/HW1/HW_Vis.png?raw=1" width = "500px" height="100px"/>

### Accessing the data

The data should be accessed from sklearn. In this section we show an example of code for accessing the documents and the document classes.

In [1]:
# Fetching the data
from sklearn.datasets import fetch_20newsgroups
import numpy as np
newsgroups = fetch_20newsgroups(subset='test')

# getting the topic ids
topic_idx = np.array(newsgroups.target, dtype=int)

# getting the unique topic names
topic_names = np.array(newsgroups.target_names)

# getting the list of documents
documents = list(newsgroups.data)

# getting the list of topics (in the same order as documents)
topics = list(topic_names[topic_idx])

These are the 20 topics in the dataset:

In [2]:
topic_names

array(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc',
       'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
       'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
       'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt',
       'sci.electronics', 'sci.med', 'sci.space',
       'soc.religion.christian', 'talk.politics.guns',
       'talk.politics.mideast', 'talk.politics.misc',
       'talk.religion.misc'], dtype='<U24')

The documents and document topics are assigned to the variables *documents* and *topics*. We print some document examples below.

## count word frequence
this function is used to count the word frequences, I will split the word such as  I'll to "I" and "ll"

For this dunction, it will return the array of top k words and the array of each word's frequences in accending order

In [4]:
import operator

def count_word_frequence(document,k):
  doc = document
  ##create a dist
  data = {}
  words = doc.replace("(",' ').replace(")",' ').replace("'",' ').replace("*",' ').replace(".",' ').replace("/",' ').replace("-",' ').replace(":",' ').replace("<",' ').replace(">",' ').replace("[",' ').replace("]",' ').replace("!",' ').replace("?",' ').replace(",",' ').strip("\n").lower()
  ##split the text
  words1 = words.split()
  ##count text
  for word in words1:
    if word in data:
      data[word] +=1
    else:
      data[word] = 1
  ###
  #sorted the dist by value in acending order
  sorted_fre = np.array(sorted(data.items(), key=operator.itemgetter(1),reverse=True))

  num_of_words = sorted_fre.shape[0]
  fre = np.zeros(num_of_words)
  words = []

  for i in range(0,num_of_words):
    fre[i] = int(sorted_fre[i][1])
    w = sorted_fre[i][0]
    words.append(w)
    
  words = np.array(words)
  

  return words[:k],fre[:k]



In [15]:
words,words_fre = count_word_frequence(document=documents[1],k=5)
print(words)
print(words_fre)

['i' 'face' 'rick' 'miller' 'x']
[10.  4.  3.  3.  3.]


## drop down menu

In [6]:
from IPython.display import display, HTML
import json

In [7]:
%%javascript
require.config({
    paths: {
        d3: "https://d3js.org/d3.v6.min"
     }
});

require(["d3"], function(d3) {
    window.d3 = d3;
});

<IPython.core.display.Javascript object>

https://www.d3-graph-gallery.com/graph/line_select.html

http://bl.ocks.org/williaster/10ef968ccfdc71c30ef8

this two websites will teach you how to make it

In [8]:
from string import Template

In [21]:
topic_names_JS = topic_names.tolist()
words_JS = topic_names_JS
words_fre = {}
print(topic_names_JS)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [22]:
template = Template("""
<html>
<head>
</head>
<body>
<div id='vis-container'>$words_fre</div>
<script>
    var topic_names = $topic;
    var words = $words;
    var fre = $words_fre;
    console.log(words);
    console.log(fre);

    var vis = function (wordsMap) {
        //var initialData = wordsMap
        var dropdown = d3.select("#vis-container")
            .insert("select", "svg");
        //.on("change", dropdownChange);

        dropdown.selectAll("option")
            .data(topic_names)
            .enter()
            .append("option")
            .attr("value", function (d, i) {
                return $topic[i] + $topic[i].slice(1, $topic[i].length);
                
            })
            .text(function (d, i) {
                return $topic[i];
            });

        //var initialData = fre;
        //updateBars(initialData);
    };

    var readData = function (data) {
        var wordsMap = {};
        data.forEach(function (d, i) {
            var w = words[i];

            wordsMap[w] = fre[i];
            console.log(w);
            console.log(wordsMap[w]);


        });


        vis(wordsMap);
    };

    readData(words);   
  
</script>
</body>

</html>
""")
my_html = template.safe_substitute(topic = topic_names_JS, words=words_JS,words_fre=words_fre)
display(HTML(my_html))